In [1]:
import json

import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

from price_net.schema import PriceAssociationScene
from price_net.attribution.extractors import create_attribution_extractor


from __future__ import annotations

import os
import csv

# from vertexai.generative_models import GenerativeModel, Part, Image

ImportError: cannot import name 'genai' from 'google' (unknown location)

In [ ]:
import os

# Set Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/lgreen/projects/gpa/google_auth.json"

# Set Google Cloud project and location (required for Vertex AI)
# Replace with your actual project ID and preferred location
os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id"  # Replace with your project
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"     # Or your preferred region

In [ ]:
while True:
    dataset_dir = Path(input("Input the dataset directory: "))
    if dataset_dir.exists() and dataset_dir.is_dir():
        break
    print("Invalid dataset directory. Please try again.")

In [ ]:
while True:
    upc_csv_path = Path(input("Input the path to the UPC to Product CSV file: "))
    if upc_csv_path.exists() and upc_csv_path.is_file():
        break
    print("Invalid UPC CSV file path. Please try again.")

In [ ]:
with open(dataset_dir / "raw_price_scenes.json", "r") as f:
   scenes = [PriceAssociationScene(**scene) for scene in json.load(f)]

# New method

In [ ]:
# Evaluate the extracted attributions
from price_net.scripts.evaluate_attributions import evaluate, AttributionEvaluationConfig

def create_evaluation_config(dataset_dir, output_dir, attributions_file):
    """Create evaluation config for the extracted attributions"""
    return AttributionEvaluationConfig(
        dataset_dir=Path(dataset_dir),
        results_dir=Path(output_dir) / "evaluation",
        cached_attributions_path=Path(attributions_file)
    )

# Create config for evaluation
eval_config = create_evaluation_config(
    dataset_dir=dataset_dir,
    output_dir="/home/lgreen/projects/price-net/output",
    attributions_file=f"/home/lgreen/projects/price-net/output/all_attributions.json"
)

# Run evaluation
print("Running evaluation...")
try:
    evaluate(eval_config)
    print("Evaluation completed! Check the results in:", eval_config.results_dir)
except Exception as e:
    print(f"Evaluation failed: {e}")

# Optional: Load and display the metrics
metrics_file = eval_config.results_dir / "attribution_metrics.yaml"
if metrics_file.exists():
    import yaml
    with open(metrics_file, 'r') as f:
        metrics = yaml.safe_load(f)
    print("\n=== Final Metrics ===")
    for key, value in metrics.items():
        if value is not None:
            print(f"{key.capitalize()}: {value:.4f}")
        else:
            print(f"{key.capitalize()}: N/A")

In [ ]:
upc_to_product_name = dict()
with open(upc_csv_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        upc, desc = row
        if upc == desc:
            desc = "unavailable"
        upc_to_product_name[upc] = desc

# Quick test - run on a single scene first
print("=== Quick Test on Single Scene ===")

def get_products_from_scene(scene: PriceAssociationScene, upc_to_name_mapping: dict) -> list[tuple]:
    """Get products list for the attribution extractor"""
    products = []
    
    all_upcs = list(scene.products.values())
    unique_upcs = set(all_upcs)
    for upc in unique_upcs:
        product_name = upc_to_name_mapping.get(upc, "unavailable")
        products.append((upc, product_name))
        
    return products

# Create extractor
extractor = create_attribution_extractor("../configs/attribution/extractors/end-to-end-gemini.yaml")

# Pick a test scene
test_scene = scenes[1]
print(f"Testing on scene: {test_scene.scene_id}")

# Get products for this scene
products = get_products_from_scene(test_scene, upc_to_product_name)


print(f"Products found: {len(products)}")
for upc, name in products:
    print(f"  - {upc}: {name}")

# Get image path
image_path = dataset_dir / "images" / test_scene.scene_id
print(f"Image path: {image_path}")

# Extract attributions
try:
    attributions = extractor(image_path, products, test_scene.scene_id)
    print(f"\n✅ Success! Extracted {len(attributions)} attributions:")
    
    for attr in attributions:
        print(f"  - UPC {attr.upc}: {attr.price.price_text} ({attr.price.price_type})")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

# Old method

In [ ]:
# train_dataset_dir = dataset_dir / "train"
# train_products_df = pd.read_csv(train_dataset_dir / "product_boxes.csv", index_col="attributionset_id")
# train_dataset = PriceAssociationDataset(root_dir=train_dataset_dir)

# val_dataset_dir= dataset_dir / "val"
# val_products_df = pd.read_csv(val_dataset_dir / "product_boxes.csv", index_col="attributionset_id")
# val_dataset = PriceAssociationDataset(root_dir=val_dataset_dir)

# test_dataset_dir= dataset_dir / "test"
# test_products_df = pd.read_csv(test_dataset_dir / "product_boxes.csv", index_col="attributionset_id")
# test_dataset = PriceAssociationDataset(root_dir=test_dataset_dir)

/tmp/ipykernel_543659/2758228553.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  train_products_df = pd.read_csv(train_dataset_dir / "product_boxes.csv", index_col="attributionset_id")
Processing scenes: 100%|██████████| 489/489 [00:02<00:00, 169.22it/s]


In [ ]:
# class PricingPromptGenerator:
#     """An object to generate multimodal LLM prompts to extract pricing information from a keyframe.

#     Attributes:
#         use_examples (bool): Whether/not to include worked examples in the pricing prompt. Examples are recommended for best performance.
#         example_img_dir (str | None, optional): Directory to store example images in (for supporting pricing prompts). Defaults to None.
#         pricing_example (list[glm.Part]): Subcomponent of all pricing prompts that shows the model examples.

#     Raises:
#         ValueError: If `use_examples` is `True`, but no `example_img_dir` is provided.
#     """

#     def __init__(self, use_examples: bool, example_img_dir: str | None = None, upc_csv_path: str | None = None):
#         if use_examples and example_img_dir is None or upc_csv_path is None:
#             raise ValueError(
#                 "Must specify directory to store example images if `use_examples` is True."
#             )

#         self.use_examples = use_examples
#         # if example_img_dir is not None:
#         #     if not os.path.exists(example_img_dir):
#         #         os.makedirs(example_img_dir)
#         self.example_img_dir = example_img_dir
#         # self._client = None
#         # load csv file with UPCs and product names to dictionary
#         self._upc_to_product_name = dict()
#         with open(upc_csv_path, newline='') as csvfile:
#             reader = csv.reader(csvfile)
#             for row in reader:
#                 upc, desc = row
#                 if upc == desc:
#                     desc = "unavailable"
#                 self._upc_to_product_name[upc] = desc
#         self._cached_example_img_1 = None
#         self._cached_example_img_2 = None


#     def generate_pricing_prompt(self, scene, dataset_dir_, products_df) -> list[str | Image] | None:
#         """Return a multimodal LLM prompt to extract pricing information from the given keyframe.

#         Args:
#             keyframe (Keyframe): The keyframe to extract pricing information from.

#         Returns:
#             list[str | Image] | None: The prompt to send to the model (as an ordered list of contents).
#                                If the keyframe's image is not downloaded, returns `None`.
#         """
#         logging.info(f"Building a pricing prompt for keyframe {scene.scene_id}")
#         local_path = products_df.loc[scene.scene_id]["local_path"]
#         if isinstance(local_path, pd.Series):
#             local_path = local_path.values[0]
#         image_path = os.path.join(dataset_dir_, local_path)
#         print(f"Trying to load image: {image_path}")
#         try:
#             scene_image = Part.from_image(Image.load_from_file(image_path))
#         except FileNotFoundError:
#             print(f"Image for keyframe {scene.scene_id} not found at {image_path}.")
#             logging.info(
#                 f"Image for keyframe {scene.scene_id} failed to download. Cannot build pricing prompt."
#             )
#             return None

#         # UPC_values will contain the UPC values for each product in the graph.
#         print("getting UPC values")
#         UPC_values = []
#         for bbox_id, bbox in scene.bboxes.items():
#         # for node_idx, bbox_id in enumerate(scene.bbox_ids):
#             filtered = test_products_df.loc[test_products_df["prod_bbox_id"] == bbox_id, "ml_label_name"]
#             if not filtered.empty:
#                 UPC_values.append(filtered.iloc[0])
#             else:
#                 UPC_values.append(None)
#         UPC_set = set([item for item in UPC_values if item is not None])
#         print("got UPC's")

#         products = [
#             (upc, self._upc_to_product_name.get(upc, "unavailable"))  # upc and product description
#             for upc in UPC_set
#         ]
        
#         print("got products")
#         prompt_products = f"""
#         Products to get prices for:
#             {products}
#         """


#         worked_examples = self.pricing_examples if self.use_examples else []
#         return [scene_image, Part.from_text(self._prompt_intro)] + worked_examples + [Part.from_text(prompt_products)]

#     @property
#     def pricing_examples(self) -> list[str | Image]:
#         example_text_1 = """
#         Here is a helpful example. Let's suppose we're looking for prices for the following products (given as a tuple with the UPC value and the product) in this image:
#         [
#             (435, Waterloo Sparkling Water Cranberry 8-12oz Cans"),
#             (801, "Waterloo Sparkling Water Mango 8-12oz Cans")
#         ]
#         1. In this image, there is only one price tag visible, reading "4.49". This must be a standard price, and the amount is $4.49.
#         Since it says "Waterloo Sparkling Water" on the price tag, we can assume it is associated with all the products in the display
#         (Waterloo Sparkling Water Cranberry and Mango 8-12oz Cans).
#         2. We now structure what we've learned as a JSON. It should always be a list of JSON objects,
#         where each price tag is an object with the price type, currency, amount, and a list of UPC's associated with that price tag.
#         [
#             {
#                 "price_type": "STANDARD",
#                 "currency": "$",
#                 "total_price": 4.49,
#                 "UPC_list": ["435", "801"]
#             }
#         ]
#         """
#         example_text_2 = """
#         This example is a bit more complicated, but should be instructive. Let's suppose we're looking for prices for the following products in this image:
#         [
#             (107, "Coca-Cola Orange Cream 20 fl oz"),
#             (129, "Dr Pepper Black Cherry 20 fl oz"),
#         ]

#         1. In this image, there are multiple price tags visible. They all read "2/$3". This is a bulk offer price, and the quantity is 2 and the total price is $3. 
#         Since each stand containing sodas has its own price tag, we can assume that the price applies to all sodas in that stand.
#         2. We now structure what we've learned as a JSON. It should always be a list of JSON objects,
#         where each price tag is an object with the price type, currency, amount, and a list of UPC's associated with that price tag.
#         [
#             {
#                 "price_type": "BULK_OFFER",
#                 "currency": "$",
#                 "quantity": 2,
#                 "amount": 3.00,
#                 "UPC_list" : ["107"]
#             },
#             {
#                 "price_type": "BULK_OFFER",
#                 "currency": "$",
#                 "quantity": 2,
#                 "amount": 3.00,
#                 "UPC_list" : ["129"]
#             },
#             {
#                 "price_type": "BULK_OFFER",
#                 "currency": "$",
#                 "quantity": 2,
#                 "total_price": 3.00,
#                 "UPC_list" : ["107", "129"]
#             }
#         ]
#         """
#         reminder = """
#         Notice that in both cases, we didn't make up any prices for products that didn't have a price tag clearly visible and attributable.
#         Likewise, multiple "UPC_list"'s had the same UPC values, but we didn't have duplicates in any "UPC_list". Please do the same in your work. 
#         We value precision (when we flag a price, it's probably correct) over recall (getting all possible prices from the image). 
#         Follow all json schemas and instructions above when generating your output. The output should be able to be parsed as valid JSON.
#         """
#         return [
#             Part.from_image(self._example_image_1),
#             Part.from_text(example_text_1),
#             Part.from_image(self._example_image_2),
#             Part.from_text(example_text_2),
#             Part.from_text(reminder),
#         ]

#     def _preload(self) -> PricingPromptGenerator:
#         """A method to force-invoke all class properties that involve a download (for cacheing).

#         Returns:
#             self: For chaining method calls.
#         """
#         self._example_image_1
#         self._example_image_2
#         return self

#     @property
#     def _prompt_intro(self) -> str:
#         return """
#         I took this image in a grocery store, and need help examining the product display to determine prices associated with the products.

#         1. First, identify and read the price tags in this image.
#         2. Next, match the prices you just identified to the products I will give you. I will provide you with a list of
#         product UPC values, and most of the time the product description, as a list of tuples. If product description is "unavailable", 
#         use reasoning to determine which products in the image it belongs to. If a product does not have a price tag associated with it, 
#         do not make up a price.
#         3. Each object in the list of objects you return must match one of the following JSON schemas:

#         - For a standard price:
#         {
#             "price_type": "STANDARD",
#             "currency": "$",
#             "amount": X.XX,
#             "UPC_list": [A, B, ...]
#         }

#         - For a bulk offer:
#         {
#             "price_type": "BULK_OFFER",
#             "currency": "$",
#             "quantity": Y,
#             "amount": X.XX,
#             "UPC_list": [A, B, ...]
#         }

#         - For a buy x get y price:
#         {
#             "price_type": "BUY_X_GET_Y",
#             "currency": "$",
#             "buy": X,
#             "get": Y,
#             "amount": X.XX,
#             "UPC_list": [A, B, ...]
#         }

#         - For an unknown price:
#         {
#             "price_type": "UNKNOWN",
#             "currency": "$",
#             "UPC_list": [A, B, ...]
#         }

#         The two most confusing are BULK_OFFER and BUY_X_GET_Y. A BULK_OFFER is a price that applies to a group of products, such as "2 for $3". A BUY_X_GET_Y is a price that applies to a product 
#         when you buy a certain number of items, such as "Buy 2 Get 1 Free". For BUY_X_GET_Y, the "amount" should be a dollar amount. If it says "BUY 2 GET 1 FREE", the "amount" should be 0.00.
#         Respond with JSON only, nothing else. Don't make up prices based on general knowledge -- use only this picture. There won't always be a price tag associated with a product. 
#         There should only be one object per price tag. UPC_list shouldn't contain duplicates.
#         """

#     @property
#     def _example_image_1(self) -> Image:
#         if self._cached_example_img_1 is None:
#             local_path = os.path.join(
#                 self.example_img_dir, "0BE5C7A8-4483-469F-8C04-2FD147714C57.jpg"
#             )
#             self._cached_example_img_1 = Image.load_from_file(local_path)

#         return self._cached_example_img_1

#     @property
#     def _example_image_2(self) -> Image:
#         if self._cached_example_img_2 is None:
#             local_path = os.path.join(
#                 self.example_img_dir, "0A81A3B2-0AD4-4E81-A7DE-EFC9D8ACF006.jpg"
#             )
#             self._cached_example_img_2 = Image.load_from_file(local_path)

#         return self._cached_example_img_2

In [ ]:
# # 1. Instantiate the model (choose the right model name for your use case)
# model = GenerativeModel("gemini-2.0-flash")

# example_img_dir = os.path.join(dataset_dir, "example_images")
# upc_csv_path = os.path.join(dataset_dir, "upc_to_product_name.csv")
# # 2. In your loop, send the prompt and get the response
# # len(test_dataset)
# with open(test_dataset_dir / PriceAssociationDataset.RAW_PRICE_SCENES_FNAME) as f:
#     scenes = [PriceAssociationScene(**x) for x in json.load(f)]
# scenes = sorted(scenes, key=lambda x: x.scene_id)

# for i in range(1,2):
#     scene = scenes[i]
#     prompt_generator = PricingPromptGenerator(
#         use_examples=True,
#         example_img_dir=example_img_dir,
#         upc_csv_path=upc_csv_path
#     )
#     prompt = prompt_generator.generate_pricing_prompt(scene, test_dataset_dir, test_products_df)
#     if prompt is not None:
#         # Send prompt to the model
#         response = model.generate_content(prompt)
#         try:
#             result = response.text
#             # Must remove the first and last lines, which are not part of the JSON response
#             lines = result.splitlines()
#             remaining_lines = lines[1:len(lines) - 1]
#             result = '\n'.join(remaining_lines)

#             parsed = json.loads(result)
#             # Save to a file, e.g., one file per keyframe
#             output_path = f"/home/lgreen/projects/price-net/output/vlm_output_{scene.scene_id}.json"
#             with open(output_path, "w") as f:
#                     json.dump(parsed, f, indent=2)
#         except Exception as e:
#             print(f"Error parsing response for keyframe {scene.scene_id}: {e}")
#     else:
#         print(f"Could not generate prompt for keyframe {scene.scene_id}.")

Trying to load image: /home/lgreen/projects/price-net/data/price-attribution-scenes/test/images/14CAB38F-12D8-4EA3-91FD-A7626958E2D2.jpg
Image for keyframe 002c7fb5-dd2e-408d-85de-543657e49d9f not found at /home/lgreen/projects/price-net/data/price-attribution-scenes/test/images/14CAB38F-12D8-4EA3-91FD-A7626958E2D2.jpg.
Could not generate prompt for keyframe 002c7fb5-dd2e-408d-85de-543657e49d9f.


/home/lgreen/projects/price-net/.venv/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
# print(result)

NameError: name 'result' is not defined

In [ ]:
# prompt_generator._upc_to_product_name["860007356605"]

'Pure Artesian Water Sodium Free'

In [ ]:
# graph: DetectionGraph = test_dataset[0]
# UPC_values = []
# for node_idx, bbox_id in enumerate(graph.bbox_ids):
#     filtered = test_products_df.loc[test_products_df["prod_bbox_id"] == bbox_id, "ml_label_name"]
#     if not filtered.empty:
#         UPC_values.append(filtered.iloc[0])
#     else:
#         UPC_values.append(None)
# UPC_set = set([item for item in UPC_values if item is not None])
# print(len(UPC_values))
# print(len(UPC_set))

90
6
